# Pair Programming Asunciones Regresión Lineal

    En este ejercicio de pair programming seguiréis usando el dataset que seleccionasteis.


    El objetivo de este pair programming es que evaluéis si vuestro set de datos cumple todas las asunciones que se deben cumplir para hacer una regresión lineal. Recordamos que estas asunciones son:
        1. Normalidad (ya la deberíais haber evaluado)


        2.  Homogeneidad de varianzas


        3. Independencia de las variables
    Cada asunción la deberéis testar de forma visual y analítica.